# Multi-index

Pour regrouper de données de même espèce.

# Démo : création et utilisation du multi-index des colonnes

In [ ]:
from pepper_commons import get_group_labels, new_multi_index
display(get_group_labels('Identification'))
mindex = new_multi_index(levels=['group', 'subgroup'])
display(mindex[:10])
#display(mindex['Identification'])

array(['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName'], dtype=object)

MultiIndex([('Identification',              '_', ...),
            ('Identification',              '_', ...),
            ('Identification',              '_', ...),
            ('Identification',              '_', ...),
            ('Identification',              '_', ...),
            (      'Location', 'Admin location', ...),
            (      'Location', 'Admin location', ...),
            (      'Location', 'Admin location', ...),
            (      'Location', 'Admin location', ...),
            (      'Location', 'Admin location', ...)],
           names=['group', 'subgroup', 'var'])

In [ ]:
from pepper_commons import get_data
data = get_data()
sindex = data.columns   # save the simple mono level index for further use
data.columns = mindex   # change the simple index for the multi-index
display(data.Location.Geolocation)
data.columns = sindex   # return to simple index

var,Latitude,Longitude
0,47.61220,-122.33799
1,47.61317,-122.33393
2,47.61393,-122.33810
3,47.61412,-122.33664
4,47.61375,-122.34047
...,...,...
3371,47.56722,-122.31154
3372,47.59625,-122.32283
3373,47.63644,-122.35784
3374,47.52832,-122.32431


# Mathématisation des variables (LaTeX like short names)

Les labels du jeu de données sont longs et n'aident pas à la lisibilité des tableaux, notamment dès que l'on rentre dans les phases de feature engineering et de modélisation.

La solution à ce problème est un mappage. Ici les correspondances LaTeX, label court, label d'origine ou définition, note optionnelle :
* Surfaces
    - $s$ ⚪ `'s'` ⚪ `'PropertyGFATotal'` ⚪ $s=s_e+s_i$
    - $s_e$ ⚪`'s_e'` ⚪ `'PropertyGFAParking'`
    - $s_i$ ⚪ `'s_i'` ⚪ `'PropertyGFABuilding(s)'`
    - $s_u$ ⚪ `'s_u'` ⚪ $\sum_{u_k \in U}{s_{u_k}}$
    - $U=\{u_k, k=0, ...\}$ ⚪ `'u_0', ...` ⚪ Ensemble des labels d'usages, ordonnés par surface totale décroissante.
    - $u_{\text{1st}}, u_{\text{2nd}}, u_{\text{3rd}}$ ⚪ `'u_1st', u_2nd, u_3rd` ⚪ `'LargestPropertyUseType'`, etc
    - $s_{\text{1st}}, s_{\text{2nd}}, s_{\text{3rd}}$ ⚪ `'s_1st', s_2nd, s_3rd` ⚪ `'LargestPropertyUseTypeGFA'`, etc
* Energies
    - $e$ ⚪ `'e'` ⚪ `'SiteEnergyUse(kBtu)'` ⚪ $e=e_e+e_s+e_g$
    - $e_e$ ⚪ `'e_e'` ⚪ `'Electricity(kBtu)'`
    - $e_s$ ⚪ `'e_s'` ⚪ `'SteamUse(kBtu)'`
    - $e_g$ ⚪ `'e_g'` ⚪ `'NaturalGas(kBtu)'`
    - $es$ ⚪ `'es'` ⚪ $s \times ies$
    - $ies$ ⚪ `'ies'` ⚪ `'SourceEUI(kBtu/sf)'`
* Emissions
    - $h$ ⚪ `'h'` ⚪ `'TotalGHGEmissions'`
* Intensités (rapportées à la surface) :
    - $ie$ ⚪ `'ie'` ⚪ `'SiteEUI(kBtu/sf)'` ⚪ $ie=e/s$
    - $ie_e$ ⚪ `'ie_e'` ⚪ $ie_e=e_e/s$
    - $ie_s$ ⚪ `'ie_s'` ⚪ $ie_s=e_s/s$
    - $ie_g$ ⚪ `'ie_g'` ⚪ $ie_g=e_g/s$
    - $ih$ ⚪ `'ih'` ⚪ $ih=h/s$

Les différences et rapports sont explicites, dont les facteurs constants éventuels :
* $1000 ih$ ⚪ `'1000 ih'` ⚪ `'GHGEmissionsIntensity'`
* $e_g / 100$ ⚪ `'e_g / 100'` ⚪ `'NaturalGas(therms)'`
* $e / 3.412$ ⚪ `'e_e / 3.412'` ⚪ `'Electricity(kWh)'`


Quand le rapport a une base constante et une partie variable qui peut être représentée à l'aide d'une distribution normale :
* $ie_{\text{wn}} = ie(1 + \mathcal{N}(0.5, \sigma^2))$  ⚪ `'e_wn'` ⚪ `'SiteEnergyUseWN(kBtu)'`
* $ies = ie(1 + (1/3)\mathcal{N}(3.14, 2.19.10^{-5}) + (2/3)\mathcal{N}(2.25, 0.3136))$  ⚪ `'es'` ⚪ `'SourceEUI(kBtu/sf)'`
* $ies_{\text{wn}}$ ...

Coordonnées géographiques :
* $\phi$ ⚪ `'lat.'` ⚪ `'Latitude'`
* $\lambda$ ⚪  `'long.'` ⚪ `'Latitude'`

Pour les variables qualitatives (autres que les détails d'usage déjà abordés plus haut) :
* `'btype'` ⚪ `'BuildingType'`
* `'ptype'` ⚪ `'PrimaryPropertyType'`

Localisation administrative : ...

In [ ]:
# fonction de mappage des noms de colonnes - nb : ce serait logique de l'ajouter dans struct !

# Production de struct.json (multi index et +)

Production du fichier json de description de la structure partant de sa description en GSheet.

C'est sur ce fichier que repose la gestion du multi-index et d'autres fonctionnalités associées au schéma du jeu de données.

A partir de la GSheet [P4 | Dictionnaire des données | struct](https://docs.google.com/spreadsheets/d/1gtTOd-taN9aY8sg4PGY456E2AlsMxi2W_-7kZaCSYlA)

## GSheet vers DataFrame

In [ ]:
from gspread_pandas import Spread
from pepper_commons import gsheet_to_df
spread = Spread('1gtTOd-taN9aY8sg4PGY456E2AlsMxi2W_-7kZaCSYlA')
struct = gsheet_to_df(spread, 'struct', header_rows=4)
struct.columns = struct.columns.droplevel([0, 1])
display(struct.head(5))

,id,group,subgroup,name,a,abstr_type,abstr_subtype,,domain,format,unity,astype,nan_code,nap_code,desc,comment
1,0,Identification,_,OSEBuildingID,1,N,ord,3 376,[1; 50226],,,,,,Un identifiant unique attribué à chaque propri...,
2,1,Identification,_,DataYear,1,N,ord,1,[2016],,,,,,L'année civile (janvier-décembre) représentée ...,
3,2,Identification,_,BuildingType,1,L,class,8,,,,,,,Classification des types de construction de la...,
4,3,Identification,_,PrimaryPropertyType,1,L,class,24,,,,,,,L'utilisation principale d'une propriété (par ...,
5,4,Identification,_,PropertyName,1,L,cdata,3 362,,,,,,,Nom de la propriété officielle ou commune tel ...,


## DataFrame vers JSON

In [ ]:
from pepper_commons import data_to_json
data_to_json(struct, 'struct.json')

## Chargement de struct.json comme DataFrame

In [ ]:
# pas utile pour la suite, mais permet de visualiser le df sur lequel s'appuie jpy_commons pour ses getters
import os
import pandas as pd
from pepper_commons import json_data_dir

struct = pd.read_json(os.path.join(json_data_dir, 'struct.json'), typ='frame', orient='index')
display(struct.head(5))

,id,group,subgroup,name,a,abstr_type,abstr_subtype,,domain,format,unity,astype,nan_code,nap_code,desc,comment
1,0,Identification,_,OSEBuildingID,1,N,ord,3 376,[1; 50226],,,,,,Un identifiant unique attribué à chaque propri...,
2,1,Identification,_,DataYear,1,N,ord,1,[2016],,,,,,L'année civile (janvier-décembre) représentée ...,
3,2,Identification,_,BuildingType,1,L,class,8,,,,,,,Classification des types de construction de la...,
4,3,Identification,_,PrimaryPropertyType,1,L,class,24,,,,,,,L'utilisation principale d'une propriété (par ...,
5,4,Identification,_,PropertyName,1,L,cdata,3 362,,,,,,,Nom de la propriété officielle ou commune tel ...,
